# Overview

This is an auxiliary notebook for the [main analyses](Main.ipynb), meant for documenting the data access and cleaning processes. All three streams of data (prisoner counts, prison costs, and overall populations, respectively) require additional  steps before they are ready for analysis. Some of these steps can be automated in a Python environment; certain minor adjustments must be done manually, and are also explained here.

# Table of Contents

I. [Prison Populations](#data1)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Description](#d1)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Access](#a1)
   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Cleaning](#a2)

II. [Prison Costs](#data2)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Description](#d2)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Access](#a3)
   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Cleaning](#a4)

III. [Overall Populations](#data3)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Description](#d3)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Access](#a5)
   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Cleaning](#a6)

# Prison Populations <a class="anchor" id="data1"></a>

## Description <a class="anchor" id="d1"></a>

Recall that the [Communicating With Prisoners](https://www.acrosswalls.org) project compiled counts of [Prisoners By State And Sex](https://www.acrosswalls.org/datasets/prisoners-us-state-sex-panel/) from 1880 to 2010. This is a count from federal and state correctional facilities, itself coming from the [US Bureau of Justice Statistics Prisoner Series](https://www.bjs.gov/index.cfm?ty=pbse&tid=0&dcid=0&sid=40&iid=0&sortby=&page=paging&curpg=4).

## Access <a class="anchor" id="a1"></a>

The source offers multiple access formats for the data file. To completely automate the data cleaning process (without using the Google Sheets API for Python) would require several steps. The URL of the public sheet hosted by the project would have to include the right tab information, range information (to exclude the extra mis-formatted parts of the CSV), and a specifier to return the result as a CSV file. In reality, many of these steps proved infeasible, and HTML parsers would be needed to scrape together a CSV from the raw response of a GET request. Meanwhile, the Google API route involves OAuth setup, use of the Google Console, and copy/paste operations on the data.

It is simpler to document the download of the file and any additoinal steps (programmed or manual) to ready the data. Although this means the data will be stored in-repository, inclusion of both the raw and clean files means future users will not need to perform data processing unless interested in replication.

1. We start by locating the Google Sheet hosted by the project, and visiting the relevant [panel tab](https://docs.google.com/spreadsheets/d/1V4_0T_lJPVBhKuMirhXncjWMI3C6crHgcD84qv1Y2JY/edit#gid=1100768042).

![img1](./figures/data1_snippet1.png)

2. It suffices to download this sheet only, as a CSV file, as highlighted in the picture.

![img2](./figures/data1_snippet2.png)

## Cleaning <a class="anchor" id="a2"></a>

The result is the file store in [data/raw/prisoner_counts.csv](./data/raw). From here, we can automate data processing steps. Run all these steps in order, exactly once (it is best to restart from __*1*__ if errors are made).

1. We load the file into a `pandas.DataFrame` object, taking care to handle the ill-formatted raw CSV file.

In [32]:
import pandas as pd
df_pp_raw = pd.read_csv('./data/raw/prisoner_counts.csv', skiprows=3)
df_pp_raw.head()

,year,aglev,region,geography,class,sexratio,males,females,Unnamed: 8,source and notes
0,2010,f,NaN,Federal,fedstate,14.48240,196222.0,13549.0,NaN,class definition:
1,2010,r,NaN,Midwest,fedstate,13.59031,245373.0,18055.0,NaN,"fedstate, jail, bjs-total: prisoner statistics..."
2,2010,r,NaN,Northeast,fedstate,18.73794,165100.0,8811.0,NaN,"census-total, census-fed, census-state, census..."
3,2010,r,NaN,South,fedstate,12.15678,604423.0,49719.0,NaN,NaN
4,2010,r,NaN,West,fedstate,12.41191,287944.0,23199.0,NaN,males and females are male and female in-priso...


2. Drop the right-most non-tabular columns, and the unneeded ratio and aglev columns.

In [33]:
df_pp_raw = df_pp_raw.drop(['Unnamed: 8', 'source and notes', 'sexratio', 'aglev'], axis=1)
df_pp_raw.head()

,year,region,geography,class,males,females
0,2010,NaN,Federal,fedstate,196222.0,13549.0
1,2010,NaN,Midwest,fedstate,245373.0,18055.0
2,2010,NaN,Northeast,fedstate,165100.0,8811.0
3,2010,NaN,South,fedstate,604423.0,49719.0
4,2010,NaN,West,fedstate,287944.0,23199.0


3. Regardless of other filters we apply, we need a column totalling male and female prisoner counts.

In [34]:
df_pp_raw['total'] = df_pp_raw.males + df_pp_raw.females
df_pp_raw.head()

,year,region,geography,class,males,females,total
0,2010,NaN,Federal,fedstate,196222.0,13549.0,209771.0
1,2010,NaN,Midwest,fedstate,245373.0,18055.0,263428.0
2,2010,NaN,Northeast,fedstate,165100.0,8811.0,173911.0
3,2010,NaN,South,fedstate,604423.0,49719.0,654142.0
4,2010,NaN,West,fedstate,287944.0,23199.0,311143.0


4. To preserve the table when transformation are applied, we replace certain missing `NaN` values. Regions are `NaN` when the row is an aggregate (e.g., an extraneous row for total federal count rather than for one state). We can impute this description in place of `NaN`. 

In [42]:
df_pp_raw['region'].fillna('aggregate', inplace=True)
df_pp_raw.head()

,year,region,geography,class,males,females,total
0,2010,aggregate,Federal,fedstate,196222.0,13549.0,209771.0
1,2010,aggregate,Midwest,fedstate,245373.0,18055.0,263428.0
2,2010,aggregate,Northeast,fedstate,165100.0,8811.0,173911.0
3,2010,aggregate,South,fedstate,604423.0,49719.0,654142.0
4,2010,aggregate,West,fedstate,287944.0,23199.0,311143.0


Observe that in some years, multiple totals are available from the `fedstate` source (US Bureeau of Justice Statistics) and the `census-total` source (Decennial Census). There are often discrepancies. While it is ideal to use the same source consistently, not all sources are available for all years, and the discrepancies imply that one source or the other is missing counts. To resolve years with multiple sources available, we can take the higher count, noting that there is a compromise in experimental consistency.

5. Drop the class column. Group by year, region, and geography, then perform a maximum aggregation of the count columns.

In [53]:
df_pp_raw = df_pp_raw.drop(['class'], axis=1)
df_pp_grouped = df_pp_raw.groupby(['year', 'region', 'geography']).max().reset_index()
df_pp_grouped.head()

,year,region,geography,males,females,total
0,1880,aggregate,U.S. Total,54190.0,5068.0,59258.0
1,1880,midwest,Illinois,3134.0,202.0,3336.0
2,1880,midwest,Indiana,1541.0,94.0,1635.0
3,1880,midwest,Iowa,806.0,13.0,819.0
4,1880,midwest,Kansas,1281.0,16.0,1297.0


6. At this point the data can be used in the main analyses. We will save this cleaned version as a CSV at [data/clean/prisoner_counts.csv](./data/clean).

In [56]:
df_pp_grouped.to_csv('./data/clean/prisoner_counts.csv')

# Prison Costs <a class="anchor" id="data2"></a>

## Description <a class="anchor" id="d2"></a>

Stuff.

## Access <a class="anchor" id="a3"></a>

Stuff.

Code.

## Cleaning <a class="anchor" id="a4"></a>

Manual Adjustments

Python cleaning (maybe include averaging, etc.). 

# Overall Populations <a class="anchor" id="data3"></a>

## Description <a class="anchor" id="d3"></a>

US population data comes primarily from the US Census Bureau: via the decennial Census, which filled decades with constant-growth estimates until 1960, and the yearly American Community Survey thereafter. The aggregator here is the international [World Bank Group](https://www.worldbank.org/). Specifically, its [World Development Indicators](https://datacatalog.worldbank.org/dataset/world-development-indicators) project collects vast development data for several countries. Its [aggregation of population over time in the US](https://data.worldbank.org/indicator/SP.POP.TOTL?locations=US) is cited as a direct combination of the Census Bureau data. State-level granularity is thus far not offered by any known aggregator for sufficient years, and impractical to self-compile.

## Access <a class="anchor" id="a5"></a>

Since only a national-level series is required, there are several ways to access the World Development Indicators data. Third-party APIs can wrap the World Bank HTTP GET request support, but the Group's [online database](https://databank.worldbank.org/source/world-development-indicators#) interface is a simpler approach. It allows us to select only the small slice of data we need, and store it in-repository so future users need not re-acquire it.

1. Visit the [interface](https://databank.worldbank.org/source/world-development-indicators#) and select "World Development Indicators" for "Database".

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![img](./figures/data3_snippet1.png)

2. Select "United States" for "Country".

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ![img](./figures/data3_snippet2.png)

3. Select "Population, total" for "Series".
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![img](./figures/data3_snippet3.png)

4. For "Time", click the "select all" icon.
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![img](./figures/data3_snippet4.png)

5. Download the slice as-is.

![img](./figures/data3_snippet5.png)  
  
  

## Cleaning <a class="anchor" id="a6"></a>

The result of the __Access__ step is the raw file stored in [data/raw/overall_population.xlsx](./data/raw/), ready for cleaning. Execute in order, once (restart if errors made).

1. Load into `pandas.DataFrame` object, with attention to any mis-formatting.

In [109]:
df_op_raw = pd.read_excel('./data/raw/overall_population.xlsx', nrows=1, thousands=',')
df_op_raw.iloc[:,:10]

,,1960,1961,1962,1963,1964,1965,Unnamed: 7,1966,1967
0,"Population, total",180671000.0,183691000.0,186538000.0,189242000.0,191889000.0,194303000.0,NaN,196560000.0,198712000.0


2. Drop three erroneous columns (two invalid, one for the blank 2020 count).

In [110]:
df_op_raw = df_op_raw.drop([' ', 'Unnamed: 7', '2020'], axis=1)
df_op_raw.iloc[:, :10]

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969
0,180671000.0,183691000.0,186538000.0,189242000.0,191889000.0,194303000.0,196560000.0,198712000.0,200706000.0,202677000.0


3. Pivot the table and make the values numeric.

In [111]:
df_pivot = df_op_raw.transpose().reset_index()
df_pivot.columns = ['year', 'total_population']
df_pivot.year = df_pivot.year.astype(int)
df_pivot.total_population = df_pivot.total_population.astype(int)
df_pivot.head()

,year,total_population
0,1960,180671000
1,1961,183691000
2,1962,186538000
3,1963,189242000
4,1964,191889000


4. We save this result as a clean CSV file, ready for main analyses, in [/data/clean/overall_population.csv](./data/clean).

In [113]:
df_pivot.to_csv('./data/clean/overall_population.csv')